In [38]:
from PIL import Image, ImageEnhance
import numpy as np
import os
from os import listdir
from os.path import isfile, join
from skimage.io import imread
from skimage.transform import resize

IMG_SIZE = 128

path = 'data'
classes = next(os.walk(path))[1]
images = []
labels = []
for label, class_name in enumerate(classes):
    dirs = next(os.walk(join(path, class_name)))[1]
    for dir_name in dirs:
        files = next(os.walk(join(path, class_name, dir_name)))[2]
        for file in files:
            filename = join(path, class_name, dir_name, file)
            try:
                img = imread(filename)[:, :, :3] # skip alpha channel
                img = resize(img, (IMG_SIZE, IMG_SIZE), mode='constant', preserve_range=True)
            except:
                pass
            
            if not img.shape == (IMG_SIZE, IMG_SIZE, 3):
                print("Unexpected shape")
                continue
                
            images.append(img)
            labels.append(label)
            print(label, img.shape)
    


0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 

0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 128, 3)
0 (128, 

1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 128, 3)
1 (128, 

In [44]:
x = np.array(images)
y = np.array(labels)

y = keras.utils.to_categorical(y)

print(x.shape, y.shape)

split = 100

test_x = x[:100]
test_y = y[:100]
train_x = x[100:200]
train_y = y[100:200]

(1214, 128, 128, 3) (1214, 2)


In [47]:
import keras
from keras import applications
from keras import backend as K
from keras.models import Sequential, Model
from keras import layers
from keras.layers import Lambda, UpSampling2D, Dense, Activation, Reshape, Flatten, BatchNormalization, AtrousConvolution2D, Conv2D, MaxPooling2D, Dropout, Input, merge
from keras.optimizers import Adam
from keras.layers.convolutional import Deconvolution2D, Conv2DTranspose
from keras.callbacks import ModelCheckpoint

vgg_model = keras.applications.VGG16(include_top=False, weights=None, input_shape = (IMG_SIZE, IMG_SIZE, 3))

# set vgg layers to non trainable
for layer in vgg_model.layers:
    layer.trainable = False
        
model = Sequential([
    vgg_model,
    Flatten(),
    Dense(100, activation='relu'),
    Dense(2, activation='softmax')
])

model.summary()

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

history = model.fit(x = train_x, y = train_y, batch_size = 4, validation_split = 0.1, epochs = 10, verbose=1)

plt.plot(history.history['acc'], label='training')
plt.plot(history.history['val_acc'], label='validation')
plt.legend()
plt.show()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_4 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 100)               819300    
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 202       
Total params: 15,534,190
Trainable params: 819,502
Non-trainable params: 14,714,688
_________________________________________________________________
Train on 90 samples, validate on 10 samples
Epoch 1/10
90/90 [==============================] - 16s 174ms/step - loss: 0.0366 - acc: 0.9556 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 2/10
90/90 [==============================] - 14s 160ms/step - loss: 

KeyboardInterrupt: 